# Explore wifi dataset
consider t1, t2, t3 three domains

Convert npz to matlab

In [1]:
from scipy.io import loadmat
from scipy.io import savemat
from os.path import join
import numpy as np
np.set_printoptions(linewidth=120)

In [10]:
for dataId in range(1, 11):
    for targetId in range(1, 4):
        file_path = '../data/wifi/tot%d_dataId%d.npz' % (targetId, dataId)
        file_path_mat = '../data/wifi/tot%d_dataId%d.mat' % (targetId, dataId)
        npzfile = np.load(file_path)
        list(npzfile.keys())
        savemat(file_path_mat, mdict={'x':npzfile['x'],'y':npzfile['y']})

In [86]:
y = loadmat(file_path_mat)['y']
print y 

[[ 25.5   0. ]
 [  6.    1. ]
 [ 25.5   2. ]
 ..., 
 [ 22.5   1. ]
 [ 27.    2. ]
 [  4.5   1. ]]


My playground

In [12]:
data_path = join('../data/wifi/', 'aligned_data.mat')
data = loadmat(data_path)
y = data['Y1_new']
unique_y = np.unique(y)
print(unique_y.shape)

(19,)


In [13]:
data_path = join('../data/wifi/', 't1t2_dag.mat')
data = loadmat(data_path)
MB = data['MB'].flatten()-1
print(MB.shape)
data['gn'][np.ix_(MB, MB)].T

(18,)


array([[ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  1.,  1.,  0.,  1.,  0.,  0.,  1.,  1.],
       [ 1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.],
       [ 1.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.],
       [ 0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.],
       [ 1.,  0.,  1.,  0.,  1.,  1.,  0., -0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  1.],
       [ 0

In [14]:
data_path = join('../data/wifi/', 't2t3_dag.mat')
data = loadmat(data_path)
data['MB'].shape

(1, 19)

In [15]:
data_path = join('../data/wifi/', 't1t3_dag.mat')
data = loadmat(data_path)
data['MB'].shape

(1, 18)

In [17]:
npzfile = np.load('../data/wifi/tot1_dataId1.npz')
print(npzfile['x'].shape)
print(npzfile['y'].shape)

(2100, 17)
(2100, 2)
